In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorflow as tf
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pickle as pkl
import gc
import csv
import pandas as pd

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_path = "/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/train/train/trainX.pt"
Y_path = "/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/train/train/trainY.pt"
test_path = "/kaggle/input/csci-ua-473-intro-to-machine-learning-fall22/test/test/testX.pt"

In [3]:
class loadableData(Dataset):
    
    
    def __init__(self, xPath, yPath = None):
        X_train = torch.load(xPath)
        self.X = X_train[0:2]
        self.ID = X_train[-1]
        self.Y = yPath
        size = len(X_train[0]) ##????
        if self.Y is not None:
            Y_train = torch.load(yPath)
            self.Y = Y_train
        self.reformat(size)
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        if self.Y is None:
            return self.X[idx], self.ID[idx]
        
        return self.X[idx], self.Y[0][idx], self.ID[idx]
    
    def getData(self):
        return self.X, self.Y
    def getDataByIndex(self,index):
        return self.X[index]
    
    def reformat(self, size):
        rgb_images = self.X[0]
        rgb_images = rgb_images[:,:1,:,:,:].squeeze()
        depth_images = self.X[1]
        depth_images = depth_images[:,:1,:,:].squeeze()

        
        rgb_images = rgb_images.permute([0,2,3,1])
        depth_images = depth_images.reshape(size,224,224,1)
        self.X = torch.cat((rgb_images, depth_images), dim=3)
        
        for sample in range(len(self.X)):
            normed = self.normalize(self.X[sample])
            self.X[sample] = normed
            
    def normalize(self, sample):
        image = sample.reshape(50176,4)
        XNormed = (image - torch.mean(image, dim=0))/torch.std(image, dim=0)
        return XNormed.reshape(224,224,4)
    
    def show(self, index):
        figure = plt.figure(figsize=(20, 20))
        figure.add_subplot(1, 1, 1)
        plt.axis("off")
        current = self.X[index].reshape(224,224,4)
        plt.imshow(current[:,:,:3])
        plt.show()
        

In [4]:
train_data = loadableData(X_path,Y_path)
test_data = loadableData(test_path)
train_loader = DataLoader(train_data, batch_size = 32, shuffle=True)
test_loader = DataLoader(test_data)

In [5]:
def train(train, epoch, model, optimizer, permute_pixels=None, permutation_order=None):

    model.train()
    for batch_idx, (data, target, idx) in enumerate(train):
        # send to device
        data = data.permute([0,3,1,2])
        data, target = data.to(device), target.to(device)
        
        # permute pixels
        if permute_pixels is not None:
            data = permute_pixels(data, permutation_order)

        optimizer.zero_grad()
        output = model.forward(data)
        output = output.to(torch.float64)

        L = nn.MSELoss()
        loss = L(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, permute_pixels=None, permutation_order=None):
    model.eval()
    
    out = []
    ids = []
    for data, idx in test_loader:
        # send to device
        data = data.permute([0,3,1,2])
        data = data.to(device)
        
        # permute pixels
        if permute_pixels is not None:
            data = permute_pixels(data, permutation_order)
            
        output = model.forward(data)
        out.append(output)
        ids.append(idx[0])
    return out, ids

In [6]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()
    self.conv1 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(56*56*16, 32)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(32, 12)

  def forward(self, x):
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.maxpool1(out)
    out = self.conv2(out)
    out = self.relu2(out)
    out = self.maxpool2(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu3(out)
    out = self.fc2(out)
    return out



In [7]:
# function to count number of parameters
def get_n_params(model):
    ans = 0
    for size in [p.size() for p in model.parameters()]:
        n = 1
        for dim in size:
            n *= dim
        ans += n
    return ans

In [8]:
model_cnn = CNNModel()
model_cnn.to(device)
optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5) # use SGD with learning rate 0.01 and momentum 0.5
print('Number of parameters: {}'.format(get_n_params(model_cnn)))
perm = [0,3,1,2]
test_accuracy = []
for epoch in range(0, 107):
    train(train_loader, epoch, model_cnn, optimizer)

Number of parameters: 1607524
Train Epoch: 0 [0/3396 (0%)]	Loss: 0.015704
Train Epoch: 0 [3200/3396 (93%)]	Loss: 0.000416
Train Epoch: 1 [0/3396 (0%)]	Loss: 0.000407
Train Epoch: 1 [3200/3396 (93%)]	Loss: 0.000364
Train Epoch: 2 [0/3396 (0%)]	Loss: 0.000357
Train Epoch: 2 [3200/3396 (93%)]	Loss: 0.000294
Train Epoch: 3 [0/3396 (0%)]	Loss: 0.000334
Train Epoch: 3 [3200/3396 (93%)]	Loss: 0.000221
Train Epoch: 4 [0/3396 (0%)]	Loss: 0.000223
Train Epoch: 4 [3200/3396 (93%)]	Loss: 0.000258
Train Epoch: 5 [0/3396 (0%)]	Loss: 0.000221
Train Epoch: 5 [3200/3396 (93%)]	Loss: 0.000204
Train Epoch: 6 [0/3396 (0%)]	Loss: 0.000191
Train Epoch: 6 [3200/3396 (93%)]	Loss: 0.000219
Train Epoch: 7 [0/3396 (0%)]	Loss: 0.000300
Train Epoch: 7 [3200/3396 (93%)]	Loss: 0.000211
Train Epoch: 8 [0/3396 (0%)]	Loss: 0.000208
Train Epoch: 8 [3200/3396 (93%)]	Loss: 0.000193
Train Epoch: 9 [0/3396 (0%)]	Loss: 0.000174
Train Epoch: 9 [3200/3396 (93%)]	Loss: 0.000210
Train Epoch: 10 [0/3396 (0%)]	Loss: 0.000161
Train

In [9]:
results, ids = test(model_cnn)

In [10]:
outfile = 'submission.csv'

output_file = open(outfile, 'w')
idTitle = ['ID']
titles = ['FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
preds = []


for sample in results:
    preds.append(sample.cpu().detach().numpy())
preds = np.array(preds).squeeze()
df = pd.DataFrame.from_records(preds, columns = titles)
idx = np.array(ids)

df.insert(0, 'ID', idx)
df
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Written to csv file submission.csv
